# Bias Analysis
### Iñaki Erregue Álvarez-Buhilla

In [7]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import h5py

from watchmal.dataset.h5_dataset import H5Dataset

## Load all the data

In [16]:
test_idxs = np.load('../time_as_channel/data/indices.npy')

true_labels = np.load('../time_as_channel/data/labels.npy')

predicted_labels = np.load('../time_as_channel/data/Q+Ts/predictions.npy')
predicted_proba = np.load('../time_as_channel/data/Q+Ts/softmax.npy')

In [ ]:
# LOAD DATA + METADATA

h5_path = '/data/neutrinos/IWCD_Data/IWCD_mPMT_Short_emgp0_E0to1000MeV_digihits.h5'
dataset = H5Dataset(h5_path, is_distributed=False)
raw_h5_file = h5py.File(h5_path, "r")

data = {
    'hit_charge': [],
    'hit_time': [],
    'hit_pmt': [],
    'event_label': [],
    'num_hits_per_event' = [],
    'polar': [],
    'azimuth': [],
    'x': [],
    'y': [],
    'z': [],
    'energies': []
}

for idx in test_idxs:
    data['event_label'].append(dataset[idx]['labels'])
    data['hit_charge'] += list(dataset.event_hit_charges)
    data['hit_time'] += list(dataset.event_hit_times)
    data['hit_pmt'] += list(dataset.event_hit_pmts)
    data['num_hits_per_event'].append(len(dataset.event_hit_pmts))

data['event_hits_index'] = dataset.event_hits_index[test_idxs]

h5_relevant_keys = ['angles', 'energies', 'positions']
for feature in h5_relevant_keys:
    for idx in test_idxs:
        if feature == 'angles':
            data['polar'].append(raw_h5_file.get(feature)[idx,0])
            data['azimuth'].append(raw_h5_file.get(feature)[idx,1])
        elif feature == 'positions':
            data['x'].append(raw_h5_file.get(feature)[idx,0,0])
            data['y'].append(raw_h5_file.get(feature)[idx,0,1])
            data['z'].append(raw_h5_file.get(feature)[idx,0,2])
        else:
            data[feature].append(raw_h5_file.get(feature)[idx,0])

# Cast to ndarray
for key in data.keys():
    data[key] = np.array(data[key])
    
# Cast to pandas df
dict_to_df = {}
for key in data.keys():
    if key not in ['hit_charge', 'hit_time', 'hit_pmt', 'event_hits_index']:
        dict_to_df[key] = data[key]
        
events_df = pd.DataFrame(dict_to_df, columns=dict_to_df.keys())
events_df['event_label'] = events_df['event_label'].map({0: 'gamma', 1: 'electron'})

## Performance in different classes